In [1]:
import pandas as pd
import plotly.graph_objects as go 
import ast
import pickle

In [2]:
# Schedule exported from the hallowed Basketball Reference
# https://www.basketball-reference.com/leagues/NBA_2024_games.html
df = pd.read_csv('sched.csv')

In [3]:
team_names = list(df['Visitor/Neutral'].unique())
current_elos = {}
elo_histories = {}
for name in team_names:
    current_elos[name] = 1500
    elo_histories[name] = [1500]

In [4]:
def expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

def update_elo(rating, expected, actual, k=20):
    """
    Update the Elo rating based on game result.
    :param rating: Current Elo rating
    :param expected: Expected score
    :param actual: Actual score (1 for win, 0 for loss)
    :param k: K-factor
    :return: Updated Elo rating
    """
    return rating + k * (actual - expected)

In [5]:
for index, row in df.iterrows(): # i know iterrows sucks and vectors are better,
                                 # but it's sequential data that's updated over time
                                 
    away = row['Visitor/Neutral']
    away_score = int(row['PTS'])
    home = row['Home/Neutral']
    home_score = int(row['PTS.1'])

    home_elo = current_elos[home]
    away_elo = current_elos[away]

    if home_score > away_score:
        result = 1
    else:
        result = 0
    
    expected_home_score = expected_score(current_elos[home], current_elos[away])
    expected_away_score = expected_score(current_elos[away], current_elos[home])
    new_rating_home = update_elo(home_elo, expected_home_score, result)
    new_rating_away = update_elo(away_elo, expected_away_score, 1 - result)

    elo_histories[home].append(new_rating_home)
    elo_histories[away].append(new_rating_away)

    current_elos[home] = new_rating_home
    current_elos[away] = new_rating_away

In [6]:
TEAM_COLORS = {
    'Los Angeles Lakers': 'rgb(85,37,130)',
    'Phoenix Suns': 'rgb(29,17,96)',
    'Houston Rockets': 'rgb(206,17,65)',
    'Boston Celtics': 'rgb(0,122,51)',
    'Washington Wizards': 'rgb(0,43,92)',
    'Atlanta Hawks': 'rgb(200,16,46)',
    'Detroit Pistons': 'rgb(200,16,46)',
    'Minnesota Timberwolves': 'rgb(12,35,64)',
    'Cleveland Cavaliers': 'rgb(134,0,56)',
    'New Orleans Pelicans': 'rgb(0,22,65)',
    'Oklahoma City Thunder': 'rgb(0,125,195)',
    'Sacramento Kings': 'rgb(91,43,130)',
    'Dallas Mavericks': 'rgb(0,83,188)',
    'Portland Trail Blazers': 'rgb(224,58,62)',
    'Philadelphia 76ers': 'rgb(0,107,182)',
    'Denver Nuggets': 'rgb(13,34,64)',
    'New York Knicks': 'rgb(0,107,182)',
    'Miami Heat': 'rgb(152,0,46)',
    'Toronto Raptors': 'rgb(206,17,65)',
    'Brooklyn Nets': 'rgb(0,0,0)',
    'Los Angeles Clippers': 'rgb(200,16,46)',
    'Orlando Magic': 'rgb(0,125,197)',
    'Golden State Warriors': 'rgb(255,199,44)',
    'Chicago Bulls': 'rgb(206,17,65)',
    'Memphis Grizzlies': 'rgb(93,118,169)',
    'Indiana Pacers': 'rgb(0,45,98)',
    'Utah Jazz': 'rgb(0,43,92)',
    'San Antonio Spurs': 'rgb(196,206,211)',
    'Milwaukee Bucks': 'rgb(0,71,27)',
    'Charlotte Hornets': 'rgb(0,120,140)'
}

TEAM_ABBRS = {
    'Los Angeles Lakers': 'LAL',
    'Phoenix Suns': 'PHX',
    'Houston Rockets': 'HOU',
    'Boston Celtics': 'BOS',
    'Washington Wizards': 'WAS',
    'Atlanta Hawks': 'ATL',
    'Detroit Pistons': 'DET',
    'Minnesota Timberwolves': 'MIN',
    'Cleveland Cavaliers': 'CLE',
    'New Orleans Pelicans': 'NOP',
    'Oklahoma City Thunder': 'OKC',
    'Sacramento Kings': 'SAC',
    'Dallas Mavericks': 'DAL',
    'Portland Trail Blazers': 'POR',
    'Philadelphia 76ers': 'PHI',
    'Denver Nuggets': 'DEN',
    'New York Knicks': 'NYK',
    'Miami Heat': 'MIA',
    'Toronto Raptors': 'TOR',
    'Brooklyn Nets': 'BKN',
    'Los Angeles Clippers': 'LAC',
    'Orlando Magic': 'ORL',
    'Golden State Warriors': 'GSW',
    'Chicago Bulls': 'CHI',
    'Memphis Grizzlies': 'MEM',
    'Indiana Pacers': 'IND',
    'Utah Jazz': 'UTA',
    'San Antonio Spurs': 'SAS',
    'Milwaukee Bucks': 'MIL',
    'Charlotte Hornets': 'CHA'
}

In [7]:
x = team_names
y = [current_elos[i] for i in team_names]
bar_colors = [TEAM_COLORS[i] for i in team_names]

sorted_indices = sorted(range(len(team_names)), key=lambda i: current_elos[team_names[i]])
# Apply the same permutation to x and bar_colors
sorted_x = [team_names[i] for i in sorted_indices]
sorted_y = [current_elos[team_names[i]] for i in sorted_indices]
sorted_bar_colors = [TEAM_COLORS[team_names[i]] for i in sorted_indices]

fig = go.Figure(data=[go.Bar(
    x=sorted_x,
    y=sorted_y,
    marker_color=sorted_bar_colors # marker color can be a single color value or an iterable
)])

fig.update_layout(
    autosize=False,
    width=2000,
    height=1200,
    plot_bgcolor='white',
    yaxis=dict(
        range=[1000, None]  # Adjust minimum_y_value to the desired minimum y value
    )
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    autorange=False,
    range=[sorted_y[0]+-100, sorted_y[-1] + 50]
)

In [8]:
games = list(range(0, 83))

fig = go.Figure() # ha ha
for team, elo in elo_histories.items():
    fig.add_trace(go.Scatter(x=games, y=elo,
                             mode='lines',
                             name=TEAM_ABBRS[team],
                             line=dict(color=TEAM_COLORS[team])))
    

fig.update_layout(
    autosize=False,
    width=2000,
    height=1200,
    plot_bgcolor='white'
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

In [9]:
with open('final_elos.pkl', 'wb') as f:
    pickle.dump(current_elos, f)